<a href="https://colab.research.google.com/github/Varun2009178/book-review-sentiment/blob/main/book-review-sentiment-analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *** Sentiment Analysis of Book Reviews with Sci-kit learn***

Set Baseline Classes and Functions

In [77]:
from re import S
import random


class Sentiment:
  NEGATIVE ='NEGATIVE'
  NEUTRAL = 'NEUTRAL'
  POSITIVE = "POSITIVE"

class Review:
  def __init__(self, text, score):
    self.text = text
    self.score =  score
    self.sentiment = self.get_sentiment()


  def get_sentiment(self):
    if self.score <= 2:
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else: #Score of 4 or 5
      return Sentiment.POSITIVE


class ReviewContainer:
  def __init__(self, reviews):
    self.reviews = reviews

  def get_text(self):
    return [x.text for x in self.reviews]

  def get_sentiment(self):
    return [x.sentiment for x in self.reviews]

  def evenly_distribute(self):
    #whenever x is negative, we are keeping track of it in the negative list
    negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
    #same for positive
    positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk
    #if not, then all the negatives would come first and then the positives
    random.shuffle(self.reviews)









Load Data

In [78]:
import json

file_name = '/content/drive/MyDrive/Data for ML AI/Books_small_10000.json'


reviews = []
with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append((Review(review['reviewText'], review['overall'])))

reviews[5].text




'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

Prep Data


In [79]:
!pip install scikit-learn

In [80]:
#Splits data into training and test
from sklearn.model_selection import train_test_split

#array, test data is 33% of pop, get same things,
training, test = train_test_split(reviews, test_size = 0.33, random_state=42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

len(train_container.reviews)






6700

In [81]:
# X (being passed in) = test, #y (output) = sentiment
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()
#can use any letter (x, y, etc) because it is for each letter in the data not a predefined variable (for each letter in the text)
test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
train_y.count(Sentiment.NEGATIVE)

436


436

In [82]:
len(training) #670 (66%)
len(test) #330 (33%)

3300

Bag of Words Vectorization

In [83]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# This book is great !
# This book was so bad
vectorizer = TfidfVectorizer()
#all of the training values
#bottom hash is the way to do it in two lines
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)
#vectorizer.fit(train_x)
#train_x_vectors = vectorizer.transform(train_x)

#print(train_x[0])
#print(train_x_vectors[0].toarray())


#train_x_vectors
#train_y



## **Classification**

Linear SVM


In [84]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])




array(['POSITIVE'], dtype='<U8')

**Decision Tree**

In [85]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

**Naive Bayes**


In [86]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()

train_x_dense = train_x_vectors.toarray()
test_x_dense = test_x_vectors.toarray()

clf_gnb.fit(train_x_dense, train_y)

clf_gnb.predict(test_x_dense[0].reshape(1, -1))

array(['NEGATIVE'], dtype='<U8')

Logistic Regression

In [87]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

# **Evaluation**

Mean Accuracy

In [ ]:
print(clf_svm.score(test_x_vectors ,test_y))
print(clf_dec.score(test_x_vectors ,test_y))
print(clf_gnb.score(test_x_dense ,test_y))
print(clf_log.score(test_x_vectors ,test_y))

0.8076923076923077
0.6346153846153846
0.6610576923076923
0.8028846153846154


F1 Scores

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEGATIVE]))


#f1_score(test_y, clf_log.predict(test_x_dense), average=None, labels = #[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])


[0.80582524 0.80952381]


In [ ]:
test_set = ['I enjoyed this, 5 stars for me',
            'horrible waste of time',
            'dont buy']

new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

## **Tuning the Model (Grid Search)**

In [ ]:
from sklearn.model_selection import GridSearchCV

#when using gridSearch, checks linear and rbf kernel, as well as for the C values, choosed one for each that will be the best on the data
parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 2, 3, 7, 17, 8)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)

clf.fit(train_x_vectors, train_y)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 2, 3, 7, 17, 8),
                         'kernel': ('linear', 'rbf')})

In [ ]:
print(clf.score(test_x_vectors, test_y))


0.8173076923076923


##Saving Model

In [ ]:
import pickle

In [62]:
with open('/content/drive/MyDrive/Data for ML AI/sentiment_classifier.pkl', 'wb') as f:
  pickle.dump(clf, f)

Load Model

In [76]:
with open('/content/drive/MyDrive/Data for ML AI/sentiment_classifier.pkl', 'rb') as f:
  loaded_clf = pickle.load(f)



In [74]:
print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

O_o that ending!!! O man! This book was gritty, disrespectful, raw..and I loved every minute of it! I do hope that another book will follow! I want a HEA!! A great read though! Belle Aurora did her thing!


array(['POSITIVE'], dtype='<U8')